### Amazon Electronics Reviever Value: Data Preparation

Our goal is to assess “reviewer lifetime value” and larger trends in spend volume by analyzing reviews of Amazon purchases. 

The Amazon reviews dataset contains 233.1 million reviews across all categories of product. In our analysis we will only focus on a subset of the electronics category, which only contains about 6.7 million records (the full electronics dataset contains almost 21 million records).

The datasets are available as compressed JSON files via download at: https://nijianmo.github.io/amazon/index.html

In [1]:
# Import packages
import pandas as pd
import numpy as np
import json
import gzip
import re

# PySpark specific modules
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *


Loading these datasets into the typical Pandas workflow requires a sizable fraction of memory on our local machines and quickly becomes unwieldy. To prepare these large datasets for analysis, we leveraged distributed data processing with the PySpark API. Spark provides an analytics engine that represents a dataset as a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel:

In [ ]:
# Create Spark session
spark = SparkSession.builder.master("local[*]").appName("Milestone_I").getOrCreate()
sc = spark.sparkContext

### Transaction metadata

**Amazon Electronics transaction metadata**: Includes descriptions, price, sales-rank, brand info, and co-purchasing links for 786,868 products. Key variables include the ID of the product, the product price in US dollars, brand name and the top category to which the product belongs.

The downloaded file is in a compressed format. Compressed files can be loaded into pyspark, but won't beneifit from the data being partitioned to each core. We unzip the file with a shell script as such:

In [ ]:
# %%sh
# gzip -d meta_Electronics.json.gz
# ls -lhtr

View the first row of JSON:

In [3]:
N = 1
with open("meta_Electronics.json") as f:
    for i in range(0, N):
        print(f.readline(), end="")

{"category": ["Electronics", "Camera &amp; Photo", "Video Surveillance", "Surveillance Systems", "Surveillance DVR Kits"], "tech1": "", "description": ["The following camera brands and models have been tested for compatibility with GV-Software.\nGeoVision \tACTi \tArecont Vision \tAXIS \tBosch \tCanon\nCNB \tD-Link \tEtroVision \tHikVision \tHUNT \tIQEye\nJVC \tLG \tMOBOTIX \tPanasonic \tPelco \tSamsung\nSanyo \tSony \tUDP \tVerint \tVIVOTEK \t \n \nCompatible Standard and Protocol\nGV-System also allows for integration with all other IP video devices compatible with ONVIF(V2.0), PSIA (V1.1) standards, or RTSP protocol.\nONVIF \tPSIA \tRTSP \t  \t  \t \nNote: Specifications are subject to change without notice. Every effort has been made to ensure that the information on this Web site is accurate. No liability is assumed for incidental or consequential damages arising from the use of the information or products contained herein."], "fit": "", "title": "Genuine Geovision 1 Channel 3rd P

By copying the output above we can [generate a schema](https://preetranjan.github.io/pyspark-schema-generator/) variable for the data:

In [5]:
schema = StructType(
    [
        StructField("category", ArrayType(StringType()), True),
        StructField("tech1", StringType(), True),
        StructField("description", ArrayType(StringType()), True),
        StructField("fit", StringType(), True),
        StructField("title", StringType(), True),
        StructField("also_buy", ArrayType(StringType()), True),
        StructField("tech2", StringType(), True),
        StructField("brand", StringType(), True),
        StructField("feature", ArrayType(StringType()), True),
        StructField("rank", ArrayType(StringType()), True),
        StructField("also_view", ArrayType(StringType()), True),
        StructField("main_cat", StringType(), True),
        StructField("similar_item", StringType(), True),
        StructField("date", StringType(), True),
        StructField("price", StringType(), True),
        StructField("asin", StringType(), True),
        StructField("imageURL", ArrayType(StringType()), True),
        StructField("imageURLHighRes", ArrayType(StringType()), True),
    ]
)

The schema above is the result of the generator, and was edited to ensure specific fields are assigned the correct datatype. For example `StringType()` was added as parameter in some `ArrayType()` fields where `null` was a parameter. 

With the schema variable we quickly load the data into a spark dataframe:

In [6]:
meta_elect_df_raw = spark.read.format("json").load("meta_Electronics.json", schema=schema)
meta_elect_df = meta_elect_df_raw
# Uncomment to view schema
# meta_elect_df.printSchema()

After creating the dataframe we can view the number of partitions the data has been split into:

In [6]:
meta_elect_df.rdd.getNumPartitions()

82

We also confirm the dimensions of the dataframe:

In [7]:
print((meta_elect_df.count(), len(meta_elect_df.columns)))

(786445, 18)


Here we see that a few hundred rows are missing (minor %). Website lists 786,868 products but here we have 786,445 records.

Next we take a look at the first few rows of data:

In [8]:
meta_elect_df.show(5)

+--------------------+-----+--------------------+---+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+----------------+--------------------+----------+--------------------+--------------------+
|            category|tech1|         description|fit|               title|            also_buy|tech2|               brand|             feature|                rank|           also_view|          main_cat|similar_item|            date|               price|      asin|            imageURL|     imageURLHighRes|
+--------------------+-----+--------------------+---+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+------------+----------------+--------------------+----------+--------------------+--------------------+
|[Electronics, Cam...|     |[The following ca...|   |Genuine Geovision...

We see our expected columns. This dataset has several interesting attributes, but for now we will select the columns we need for an assessment of reviewer value:

In [9]:
cols_to_use = ["asin", "brand", "main_cat", "price", "title"]
meta_elect_df.select(cols_to_use).show(5)

+----------+--------------------+------------------+--------------------+--------------------+
|      asin|               brand|          main_cat|               price|               title|
+----------+--------------------+------------------+--------------------+--------------------+
|0011300000|           GeoVision|Camera &amp; Photo|              $65.00|Genuine Geovision...|
|0043396828|        33 Books Co.|Camera &amp; Photo|                    |Books "Handbook o...|
|0060009810|Visit Amazon's Ca...|             Books|              $11.49|      One Hot Summer|
|0060219602|Visit Amazon's Di...|             Books|.a-section.a-spac...|Hurray for Hattie...|
|0060786817|Visit Amazon's Lo...|             Books|              $13.95|sex.lies.murder.f...|
+----------+--------------------+------------------+--------------------+--------------------+
only showing top 5 rows



When looking at the price attribute above we can see that there are rows with missing values, and rows with non price values. We only want rows with prices so we will use regex to filter for values that start with a doller sign ($):

In [10]:
# Regex for anything that starts with a dollar sign ($)
expr = "\$.*"  

# Filter with expression
meta_elect_df = meta_elect_df.filter(meta_elect_df.price.rlike(expr)).select(
    cols_to_use
)
meta_elect_df.show(5)

+----------+--------------------+--------------------+------+--------------------+
|      asin|               brand|            main_cat| price|               title|
+----------+--------------------+--------------------+------+--------------------+
|0011300000|           GeoVision|  Camera &amp; Photo|$65.00|Genuine Geovision...|
|0060009810|Visit Amazon's Ca...|               Books|$11.49|      One Hot Summer|
|0060786817|Visit Amazon's Lo...|               Books|$13.95|sex.lies.murder.f...|
|0091912407|            ABBY LEE|               Books| $4.76|Girl with a One-t...|
|0132492776|     Enter The Arena|Home Audio & Theater| $7.99|Wireless Bluetoot...|
+----------+--------------------+--------------------+------+--------------------+
only showing top 5 rows



We check the dimensions again after filtering:

In [11]:
# Check dimensions
print((meta_elect_df.count(), len(meta_elect_df.columns)))

(304323, 5)


About half of the dataset remains after filtering for only rows with prices.

### Amazon Electronics Reviews 5-core  (5-core)

**Amazon Electronics Reviews 5-core dataset**: Contains 6,739,590 user reviews from 1999 to 2018. These data have been reduced to extract the k-core, such that each of the remaining users and items have k reviews each. Key variables include the ID of the reviewer, ID of the product and the time of the review.

Same as before, we begin by decompressing the file:

In [35]:
# %%sh

# gzip -d Electronics_5.json.gz
# ls -lhtr

Read the first line of JSON: 

In [11]:
N = 1
with open("Electronics_5.json") as f:
    for i in range(0, N):
        print(f.readline(), end="")

{"overall": 5.0, "vote": "67", "verified": true, "reviewTime": "09 18, 1999", "reviewerID": "AAP7PPBU72QFM", "asin": "0151004714", "style": {"Format:": " Hardcover"}, "reviewerName": "D. C. Carrad", "reviewText": "This is the best novel I have read in 2 or 3 years.  It is everything that fiction should be -- beautifully written, engaging, well-plotted and structured.  It has several layers of meanings -- historical, family,  philosophical and more -- and blends them all skillfully and interestingly.  It makes the American grad student/writers' workshop \"my parents were  mean to me and then my professors were mean to me\" trivia look  childish and silly by comparison, as they are.\nAnyone who says this is an  adolescent girl's coming of age story is trivializing it.  Ignore them.  Read this book if you love literature.\nI was particularly impressed with  this young author's grasp of the meaning and texture of the lost world of  French Algeria in the 1950's and '60's...particularly poig

Copy first row of JSON output above and [Generate Schema](https://preetranjan.github.io/pyspark-schema-generator/):

In [7]:
schema = StructType(
    [
        StructField(
            "overall", FloatType(), True
        ),  # Changed to FloatType from StringType
        StructField("vote", StringType(), True),
        StructField("verified", BooleanType(), True),
        StructField("reviewTime", StringType(), True),
        StructField("reviewerID", StringType(), True),
        StructField("asin", StringType(), True),
        StructField(
            "style", StructType([StructField("Format:", StringType(), True)]), True
        ),
        StructField("reviewerName", StringType(), True),
        StructField("reviewText", StringType(), True),
        StructField("summary", StringType(), True),
        StructField("unixReviewTime", IntegerType(), True),
    ]
)

Use schema so spark does not have to infer:

In [8]:
# 5 core electronics data
e5_core_df_raw = spark.read.json("Electronics_5.json", schema=schema)
e5_core_df = e5_core_df_raw
# Uncomment to view schema 
# e5_core_df.printSchema()

Show data:

In [14]:
e5_core_df.show(5)

+-------+----+--------+-----------+--------------+----------+-----------------+----------------+--------------------+--------------------+--------------+
|overall|vote|verified| reviewTime|    reviewerID|      asin|            style|    reviewerName|          reviewText|             summary|unixReviewTime|
+-------+----+--------+-----------+--------------+----------+-----------------+----------------+--------------------+--------------------+--------------+
|    5.0|  67|    true|09 18, 1999| AAP7PPBU72QFM|0151004714|     { Hardcover}|    D. C. Carrad|This is the best ...|      A star is born|     937612800|
|    3.0|   5|    true|10 23, 2013|A2E168DTVGE6SV|0151004714|{ Kindle Edition}|             Evy|Pages and pages o...|A stream of consc...|    1382486400|
|    5.0|   4|   false| 09 2, 2008|A1ER5AYS3FQ9O3|0151004714|     { Paperback}|           Kcorn|This is the kind ...|I'm a huge fan of...|    1220313600|
|    5.0|  13|   false| 09 4, 2000|A1T17LMQABMBN5|0151004714|     { Hardcove

Check partitions:

In [15]:
e5_core_df.rdd.getNumPartitions()

32

Check shape:

In [16]:
print((e5_core_df.count(), len(e5_core_df.columns)))

(6739590, 11)


The dataframe has expected number of rows. Now to select the columns:

In [17]:
cols_to_use = ["asin", "overall", "unixReviewTime", "reviewerID", "reviewText"]
e5_core_df = e5_core_df.select(cols_to_use)
e5_core_df.show(5)

+----------+-------+--------------+--------------+--------------------+
|      asin|overall|unixReviewTime|    reviewerID|          reviewText|
+----------+-------+--------------+--------------+--------------------+
|0151004714|    5.0|     937612800| AAP7PPBU72QFM|This is the best ...|
|0151004714|    3.0|    1382486400|A2E168DTVGE6SV|Pages and pages o...|
|0151004714|    5.0|    1220313600|A1ER5AYS3FQ9O3|This is the kind ...|
|0151004714|    5.0|     968025600|A1T17LMQABMBN5|What gorgeous lan...|
|0151004714|    3.0|     949622400|A3QHJ0FXK33OBE|I was taken in by...|
+----------+-------+--------------+--------------+--------------------+
only showing top 5 rows



With both datasets loaded we can now merge the data.

### Merge data 

We left join the data on the product ID number (`asin`), creating a new dataframe:

In [18]:
elect_df = e5_core_df.join(meta_elect_df, on="asin", how="left")
elect_df.show(5)
# print('5 core dataset: ', (e5_core_df.count(), len(e5_core_df.columns)))
# print('Joined dataset: ', (elect_df.count(), len(elect_df.columns)))

+----------+-------+--------------+--------------+--------------------+-----+------------------+-------+--------------------+
|      asin|overall|unixReviewTime|    reviewerID|          reviewText|brand|          main_cat|  price|               title|
+----------+-------+--------------+--------------+--------------------+-----+------------------+-------+--------------------+
|B000NOEDGK|    5.0|    1378339200|A195EZSQDW3E21|Few electronic pr...|Nikon|Camera &amp; Photo|$189.99|Nikon D40x 10.2MP...|
|B000NOEDGK|    5.0|    1371945600|A31AX66K0TX9RE|I got this item f...|Nikon|Camera &amp; Photo|$189.99|Nikon D40x 10.2MP...|
|B000NOEDGK|    5.0|    1369872000| A1SQ6W1WC94E4|After using my D4...|Nikon|Camera &amp; Photo|$189.99|Nikon D40x 10.2MP...|
|B000NOEDGK|    5.0|    1363132800|A2FLZMVFJS231L|I've had this cam...|Nikon|Camera &amp; Photo|$189.99|Nikon D40x 10.2MP...|
|B000NOEDGK|    5.0|    1314057600| A4EOR1UV8KU93|This is my second...|Nikon|Camera &amp; Photo|$189.99|Nikon D40x 10.

Next we check this new dataframe for missing values. (This can take a while, possibly minutes):

In [19]:
elect_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in elect_df.columns]
   ).show()

+----+-------+--------------+----------+----------+-------+--------+-------+-------+
|asin|overall|unixReviewTime|reviewerID|reviewText|  brand|main_cat|  price|  title|
+----+-------+--------------+----------+----------+-------+--------+-------+-------+
|   0|      0|             0|         0|      1380|2389218| 2389218|2389218|2389218|
+----+-------+--------------+----------+----------+-------+--------+-------+-------+



We see there are about 2.3 million rows that lack price data. Drop any rows with missing values:

In [20]:
elect_df = elect_df.na.drop()

Confirm missing values were dropped (can take a while, possibly minutes):

In [21]:
elect_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in elect_df.columns]
   ).show()

+----+-------+--------------+----------+----------+-----+--------+-----+-----+
|asin|overall|unixReviewTime|reviewerID|reviewText|brand|main_cat|price|title|
+----+-------+--------------+----------+----------+-----+--------+-----+-----+
|   0|      0|             0|         0|         0|    0|       0|    0|    0|
+----+-------+--------------+----------+----------+-----+--------+-----+-----+



Check dimensions again after dropping missing data:

In [24]:
print("# rows: ", elect_df.select('asin').count())

# rows:  4585331


4.5 million records remain (down from 6.7 million).

### Clean dates

Next we clean the `unixreviewTime` column and extract the year and month to individual columns for ease of use:

In [22]:
elect_df = (
    elect_df.withColumn(
        "unixReviewTime", from_unixtime(col("unixReviewTime"), "MM-dd-yyyy")
    )
    .withColumn("unixReviewTime", to_date(col("unixReviewTime"), "MM-dd-yyyy"))
    .withColumn("year", year("unixReviewTime"))
    .withColumn("month", month("unixReviewTime"))
)

elect_df.show(5)

+----------+-------+--------------+--------------+--------------------+--------------------+--------+------+--------------------+----+-----+
|      asin|overall|unixReviewTime|    reviewerID|          reviewText|               brand|main_cat| price|               title|year|month|
+----------+-------+--------------+--------------+--------------------+--------------------+--------+------+--------------------+----+-----+
|0446697192|    5.0|    2009-07-13|A3LXXYBYUHZWS5|Fresh from Connec...|Visit Amazon's Zo...|   Books|$17.99|Hollywood Is like...|2009|    7|
|0446697192|    5.0|    2009-07-09|A1X4L7AO1BXMHK|I don't know abou...|Visit Amazon's Zo...|   Books|$17.99|Hollywood Is like...|2009|    7|
|0446697192|    3.0|    2009-09-01|A1Y9RUTH5GG3MU|Obviously the pre...|Visit Amazon's Zo...|   Books|$17.99|Hollywood Is like...|2009|    9|
|0446697192|    4.0|    2009-08-29| AAR8E3JF9K93P|I am very happy t...|Visit Amazon's Zo...|   Books|$17.99|Hollywood Is like...|2009|    8|
|0446697192| 

### Clean Price
Using regex, we remove the dollar sign from in front of prices and convert datatype from string to numerical:

In [23]:
elect_df = elect_df.withColumn(
    "price", regexp_replace("price", "[$,]", "").cast("double")
)
# elect_df.printSchema()
elect_df.show(5)

+----------+-------+--------------+--------------+--------------------+--------------------+--------+-----+--------------------+----+-----+
|      asin|overall|unixReviewTime|    reviewerID|          reviewText|               brand|main_cat|price|               title|year|month|
+----------+-------+--------------+--------------+--------------------+--------------------+--------+-----+--------------------+----+-----+
|0446697192|    5.0|    2009-07-13|A3LXXYBYUHZWS5|Fresh from Connec...|Visit Amazon's Zo...|   Books|17.99|Hollywood Is like...|2009|    7|
|0446697192|    5.0|    2009-07-09|A1X4L7AO1BXMHK|I don't know abou...|Visit Amazon's Zo...|   Books|17.99|Hollywood Is like...|2009|    7|
|0446697192|    3.0|    2009-09-01|A1Y9RUTH5GG3MU|Obviously the pre...|Visit Amazon's Zo...|   Books|17.99|Hollywood Is like...|2009|    9|
|0446697192|    4.0|    2009-08-29| AAR8E3JF9K93P|I am very happy t...|Visit Amazon's Zo...|   Books|17.99|Hollywood Is like...|2009|    8|
|0446697192|    4.0|

Finally, we [partition](https://sparkbyexamples.com/spark/spark-read-write-dataframe-parquet-example/) and save data:

In [ ]:
# elect_df.write.partitionBy("year").parquet("electronics_cleaned.parquet")

Now the data is in a prepared state in which we can perform deeper analysis.

### Create Sample Datasets

Next we create and export sample datasets (both raw and clean merged) for the reader to review:

In [11]:
meta_elect_df_raw.limit(100).toPandas().to_json('meta_elect_df_sample.json')
e5_core_df_raw.limit(100).toPandas().to_json('e5_core_df_sample.json')
elect_df.limit(100).toPandas().to_json('clean_merged_df_sample.json')